# Offline Engine API

SGLang provides a direct inference engine without the need for an HTTP server, especially for use cases where additional HTTP server adds unnecessary complexity or overhead. Here are two general use cases:

- Offline Batch Inference
- Custom Server on Top of the Engine

This document focuses on the offline batch inference, demonstrating four different inference modes:

- Non-streaming synchronous generation
- Streaming synchronous generation
- Non-streaming asynchronous generation
- Streaming asynchronous generation

Additionally, you can easily build a custom server on top of the SGLang offline engine. A detailed example working in a python script can be found in [custom_server](https://github.com/sgl-project/sglang/blob/main/examples/runtime/engine/custom_server.py).

## Offline Batch Inference

SGLang offline engine supports batch inference with efficient scheduling.

In [1]:
# launch the offline engine

import sglang as sgl
import asyncio

llm = sgl.Engine(model_path="meta-llama/Meta-Llama-3.1-8B-Instruct")

Loading safetensors checkpoint shards:   0% Completed | 0/4 [00:00<?, ?it/s]


Loading safetensors checkpoint shards:  25% Completed | 1/4 [00:00<00:02,  1.11it/s]


Loading safetensors checkpoint shards:  50% Completed | 2/4 [00:01<00:01,  1.04it/s]


Loading safetensors checkpoint shards:  75% Completed | 3/4 [00:02<00:00,  1.52it/s]


Loading safetensors checkpoint shards: 100% Completed | 4/4 [00:03<00:00,  1.30it/s]
Loading safetensors checkpoint shards: 100% Completed | 4/4 [00:03<00:00,  1.27it/s]



...

 70%|██████▉   | 16/23 [00:03<00:01,  4.53it/s]

 78%|███████▊  | 18/23 [00:04<00:01,  4.77it/s]

 87%|████████▋ | 20/23 [00:04<00:00,  5.24it/s]

 96%|█████████▌| 22/23 [00:04<00:00,  5.24it/s]

100%|██████████| 23/23 [00:05<00:00,  4.46it/s]


### Non-streaming Synchronous Generation

In [2]:
prompts = [
    "Hello, my name is",
    "The president of the United States is",
    "The capital of France is",
    "The future of AI is",
]

sampling_params = {"temperature": 0.8, "top_p": 0.95}

outputs = llm.generate(prompts, sampling_params)
for prompt, output in zip(prompts, outputs):
    print("===============================")
    print(f"Prompt: {prompt}\nGenerated text: {output['text']}")

Prompt: Hello, my name is
Generated text:  Samantha (Sam) and I’m a mental health nurse. I have a keen interest in mental health, psychology, and personal development. As a nurse, I have worked with a range of clients who have experienced trauma, anxiety, depression, and other mental health issues. My passion is to help people overcome their struggles and live a more fulfilling life.
My blog is a space where I share my thoughts, experiences, and knowledge on mental health and wellness. I aim to provide helpful information, advice, and insights that can support you in your own journey towards better mental health and self-care.
I believe that mental health is just as important as
Prompt: The president of the United States is
Generated text:  often called the leader of the free world, but President Obama's appearance on Saturday Night Live this weekend suggests a different reality.
President Obama's appearance on Saturday Night Live was a first for a sitting president in 15 years, and it

### Streaming Synchronous Generation

In [3]:
prompts = [
    "Hello, my name is",
    "The capital of France is",
    "The future of AI is",
]
sampling_params = {"temperature": 0.8, "top_p": 0.95}

print("\n=== Testing synchronous streaming generation ===")

for prompt in prompts:
    print(f"\nPrompt: {prompt}")
    print("Generated text: ", end="", flush=True)

    for chunk in llm.generate(prompt, sampling_params, stream=True):
        print(chunk["text"], end="", flush=True)
    print()


=== Testing synchronous streaming generation ===

Prompt: Hello, my name is
Generated text: 

 A

beer

,

 I

...

 AI

 may

 automate

 certain

 tasks

### Non-streaming Asynchronous Generation

In [4]:
prompts = [
    "Hello, my name is",
    "The capital of France is",
    "The future of AI is",
]

sampling_params = {"temperature": 0.8, "top_p": 0.95}

print("\n=== Testing asynchronous batch generation ===")


async def main():
    outputs = await llm.async_generate(prompts, sampling_params)

    for prompt, output in zip(prompts, outputs):
        print(f"\nPrompt: {prompt}")
        print(f"Generated text: {output['text']}")


asyncio.run(main())


=== Testing asynchronous batch generation ===



Prompt: Hello, my name is
Generated text:  Shirley. I am a 30 year old woman, from the UK. I love animals, especially cats and dogs. I am a bit of a movie buff, and enjoy watching a good film on my nights off. I am also a big fan of music, and have been to a lot of concerts in my lifetime. I am looking to meet new people, and make some new friends.
I am easy going, and enjoy trying new things. I am also a bit of a foodie, and love trying out new restaurants and cuisines. I am also quite fit, and enjoy going to the gym and taking part in sports.


Prompt: The capital of France is
Generated text:  a city steeped in history and culture, and it is one of the most popular tourist destinations in the world. Paris is famous for its stunning architecture, art, fashion, and cuisine, and its picturesque streets and landmarks are a photographer's dream.
The Eiffel Tower is an iconic symbol of Paris and one of the most recognizable landmarks in the world. It was built for the 1889 World's Fair an

### Streaming Asynchronous Generation

In [5]:
prompts = [
    "Hello, my name is",
    "The capital of France is",
    "The future of AI is",
]
sampling_params = {"temperature": 0.8, "top_p": 0.95}

print("\n=== Testing asynchronous streaming generation ===")


async def main():
    for prompt in prompts:
        print(f"\nPrompt: {prompt}")
        print("Generated text: ", end="", flush=True)

        generator = await llm.async_generate(prompt, sampling_params, stream=True)
        async for chunk in generator:
            print(chunk["text"], end="", flush=True)
        print()


asyncio.run(main())


=== Testing asynchronous streaming generation ===

Prompt: Hello, my name is
Generated text: 

 Tom

.

 I

'm

...

 Enh

ancing

 education

:

 AI

In [6]:
llm.shutdown()